In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from clean_data import clean_data, prepare_data
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize

In [14]:
# Read files
train = pd.read_csv('../cars-competition/data/cars_train.csv', index_col='Id')
submission = pd.read_csv('../cars-competition/data/cars_test.csv', index_col='Id')

In [15]:
# Clean Data
train = clean_data(train)
train = train[train['year']>=1930]
submission = clean_data(submission)

Cleaning cylinders...
Cleaning condition...
Cleaning odometer...
Cleaning drive...
Cleaning size...
Cleaning manufacturer...
Cleaning fuel...
Cleaning transmission...
Cleaning title status...
Data cleaning complete!
Cleaning cylinders...
Cleaning condition...
Cleaning odometer...
Cleaning drive...
Cleaning size...
Cleaning manufacturer...
Cleaning fuel...
Cleaning transmission...
Cleaning title status...
Data cleaning complete!


In [16]:
# Prepare Data
# columns = ['year','manufacturer','condition','cylinders','fuel','odometer','title_status','transmission','drive','size']
columns = ['year','manufacturer','condition','cylinders','odometer','title_status','transmission','size','lat','long']
X, y = prepare_data(train,columns,typ='train')
X_sub, _ = prepare_data(submission,columns,typ='test')

In [17]:
# Normalize columns
norm = ['year','condition','cylinders','odometer','lat','long']
X[norm] = normalize(X[norm])
X_sub[norm] = normalize(X_sub[norm])

In [18]:
# Feature reduction PCA
n_components = [5,10,15,20,25,30,35,40]
#n_components = [5,10,15]
for n in n_components:
    exec('pca{0} = PCA(n_components = {0})'.format(n))
    exec('X_new{0} = pca{0}.fit_transform(X)'.format(n))

In [19]:
# Split Train and Test

for n in n_components:
    exec('X_train{0}, X_test{0}, y_train{0}, y_test{0} = train_test_split(X_new{0},y,test_size=0.2,random_state=200)'.format(n))

In [20]:
# Linear Regression
for n in n_components:
    exec('lin_reg{0} = LinearRegression()'.format(n))
    exec('lin_reg{0}.fit(X_train{0},y_train{0})'.format(n))

In [21]:
# Predict
for n in n_components:
    exec('y_pred{0} = lin_reg{0}.predict(X_test{0})'.format(n))

In [22]:
# Check error
error = {}
for n in n_components:
    exec('error[{0}] = mean_squared_error(y_test{0}, y_pred{0})'.format(n))

In [23]:
# Check for optimal feature number for PCA: 25
error

{5: 64844872333188.97,
 10: 64844978964643.18,
 15: 64842671667769.17,
 20: 64842164645063.96,
 25: 64843875373316.91,
 30: 64843845380356.0,
 35: 64844429677195.24,
 40: 64844570644491.945}

In [ ]:
# 153496416809193.97
# 64842164645063.96

In [27]:
# Train Model with all data
lin_reg_sub = LinearRegression()
lin_reg_sub.fit(X_new20,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [28]:
# Try optimal Linear Regression for submission
# PCA
X_sub20 = pca20.transform(X_sub)
# Predict
y_sub20 = lin_reg_sub.predict(X_sub20)
y_sub20

array([ 47514.14960155, 153585.85714717,  12847.83604174, ...,
       144273.50745922,  52179.91960933, 148281.04176366])

In [29]:
# Getting absolute values
y_sub20 = abs(y_sub20)
y_sub20

array([ 47514.14960155, 153585.85714717,  12847.83604174, ...,
       144273.50745922,  52179.91960933, 148281.04176366])

In [30]:
# Prepare for submission
sub20 = submission
sub20['price'] = y_sub20
sub20 = sub20['price'] 
sub20.to_csv('../cars-competition/data/sub20_ln_yeargte1930.csv', header=True, index=True)